In [ ]:
#%pip install -r requirements.txt
#%pip install huggingface_hub
#%pip install transformers torch

Existem 5 tipos diferentes de mensagens:

- `HumanMessage`: Isso representa uma mensagem do usuário. Geralmente consiste apenas de conteúdo.

- `AIMessage`: Isso representa uma mensagem do modelo. Pode ter additional_kwargs incluídos - por exemplo, tool_calls se estiver usando chamadas de ferramentas da OpenAI.

- `SystemMessage`: Isso representa uma mensagem do sistema, que indica ao modelo como se comportar. Geralmente consiste apenas de conteúdo. Nem todo modelo suporta isso.

- `FunctionMessage`: Isso representa o resultado de uma chamada de função. Além do papel e conteúdo, esta mensagem tem um parâmetro de nome que transmite o nome da função que foi chamada para produzir este resultado.

- `ToolMessage`: Isso representa o resultado de uma chamada de ferramenta. Isso é distinto de uma Mensagem de Função a fim de corresponder aos tipos de mensagens de função e ferramenta da OpenAI. Além do papel e conteúdo, esta mensagem tem um parâmetro tool_call_id que transmite o id da chamada à ferramenta que foi feita para produzir este resultado.

### Importação da API do Gemini

In [ ]:
import getpass
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_community.chat_models.huggingface import ChatHuggingFace
from langchain_community.llms.huggingface_endpoint import HuggingFaceEndpoint
from langchain_openai import OpenAI
from langchain_openai.chat_models import ChatOpenAI

from dotenv import load_dotenv
load_dotenv()

In [ ]:
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")
    

### Configuração modelo

In [ ]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2
)

### Teste de configuração do modelo

In [ ]:
messages = [
    ("system", "Você é um assistente que traduz do inglês para o francês. Traduza o seguinte."),
    ("human", "I love programming.")
]
ai_msg = llm.invoke(messages)
print(ai_msg.content)

### Chamadas simultâneas

In [ ]:
perguntas = [
	'O que é o céu?',
	'O que é o mar?',
	'Como podemos ver as estrelas?'
]

llm.batch(perguntas)

### Chat Models

In [ ]:
mensagens = [
	SystemMessage(content='Você é um assistente que conta piadas.'),
	HumanMessage(content='Conte uma piada de um velho português')
]
llm.invoke(mensagens)

In [ ]:
#Conteúdo da mensagem
print(llm.invoke(mensagens).content)

In [ ]:
#Metadados da mensagem
print(llm.invoke(mensagens).response_metadata)

In [ ]:
#Uma maneira mais interessante de mostrar o conteúdo do chat
mensagens = [
	SystemMessage(content='Você é um assistente que conta piadas.'),
	HumanMessage(content='Conte uma piada de um velho português')
]
for trecho in llm.stream(mensagens):
	print(trecho.content, end='')

### Prompt few-shot

In [ ]:
mensagens = [
	HumanMessage(content='Quando é 1 + 1?'),
	AIMessage(content='2'),
	HumanMessage(content='Quando é 10 * 5?'),
	AIMessage(content='50'),
	HumanMessage(content='Quando é 10 + 3?'),
]

print(llm.invoke(mensagens).content)

### Utilização de outros modelos

In [ ]:
if not os.getenv("HUGGINGFACEHUB_API_TOKEN"):
    os.environ["HUGGINGFACEHUB_API_TOKEN"] = getpass.getpass("Enter your token: ")

In [ ]:
modelo = 'mistralai/Mixtral-8x7B-Instruct-v0.1'
llm = HuggingFaceEndpoint(repo_id=modelo)
chat = ChatHuggingFace(llm=llm)

### Debugando as LLMs

In [ ]:
import langchain
langchain.debug = True
llm.invoke(mensagens)

### Caching

In [ ]:
chat = ChatOpenAI (model='gpt-3.5-turbo-0125')

In [20]:
mensagens = [
	SystemMessage(content='Você um especialista em matemática.'),
	HumanMessage(content='Quanto é a soma de 1 + 1?')
]

In [21]:
from langchain.cache import InMemoryCache
from langchain.globals import set_llm_cache

set_llm_cache(InMemoryCache())

In [22]:
#Executando pela primeira vez.
%%time
chat.invoke(mensagens)

[llm/start] [llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: Você um especialista em matemática.\nHuman: Quanto é a soma de 1 + 1?"
  ]
}
[llm/end] [llm:ChatOpenAI] [3.39s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "A soma de 1 + 1 é igual a 2.",
            "additional_kwargs": {
              "refusal": null
            },
            "response_metadata": {
              "token_usage": {
                "completion_tokens": 14,
                "prompt_tokens": 32,
                "total_tokens": 46,
                "completion_tokens_details": {
                 

AIMessage(content='A soma de 1 + 1 é igual a 2.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 32, 'total_tokens': 46, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--a6afb917-2c84-444a-91c8-7a6ee61b6555-0', usage_metadata={'input_tokens': 32, 'output_tokens': 14, 'total_tokens': 46, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [23]:
#Executando pela segunda vez, com o cache ativado.
%%time
chat.invoke(mensagens)

[llm/start] [llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: Você um especialista em matemática.\nHuman: Quanto é a soma de 1 + 1?"
  ]
}
[llm/end] [llm:ChatOpenAI] s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "A soma de 1 + 1 é igual a 2.",
            "additional_kwargs": {
              "refusal": null
            },
            "response_metadata": {
              "token_usage": {
                "completion_tokens": 14,
                "prompt_tokens": 32,
                "total_tokens": 46,
                "completion_tokens_details": {
                  "acc

AIMessage(content='A soma de 1 + 1 é igual a 2.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 32, 'total_tokens': 46, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--a6afb917-2c84-444a-91c8-7a6ee61b6555-0', usage_metadata={'input_tokens': 32, 'output_tokens': 14, 'total_tokens': 46, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### Cache usando SQLite

In [24]:
from langchain.cache import SQLiteCache
from langchain.globals import set_llm_cache

set_llm_cache(SQLiteCache(database_path='./arquivos/langchain_cache_db.sqlite'))

In [26]:
%%time
chat.invoke(mensagens)

[llm/start] [llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: Você um especialista em matemática.\nHuman: Quanto é a soma de 1 + 1?"
  ]
}
[llm/end] [llm:ChatOpenAI] [2.20s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "generation_info": {
          "finish_reason": "stop",
          "logprobs": null
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "A soma de 1 + 1 é igual a 2.",
            "additional_kwargs": {
              "refusal": null
            },
            "response_metadata": {
              "token_usage": {
                "completion_tokens": 14,
                "prompt_tokens": 32,
                "total_tokens": 46,
                "completion_tokens_details": {
                 

AIMessage(content='A soma de 1 + 1 é igual a 2.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 32, 'total_tokens': 46, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--b0075ab6-f081-4aec-b88d-d7b3dfc9cca6-0', usage_metadata={'input_tokens': 32, 'output_tokens': 14, 'total_tokens': 46, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})